# OpenVPN 服务器安装教程

---

OpenVPN 相比较其他的VPN而言，个人觉得是最好安装部署与使用的一个VPN。最近在腾讯云领了一个试用服务器，于是来折腾了一下，并把一些步骤记录下。主要就是要配置openvpn、证书以及防火墙。

In [18]:
digraph{
    rankdir=LR
    openvpn
    证书
    防火墙
    openvpn->证书->防火墙->openvpn
}

## 目录：

- [运行环境](#运行环境)
- [安装](#安装)
- [证书和密钥](#证书和密钥)
- [防火墙配置](#防火墙配置)

## 运行环境

- 操作系统：ubuntu16.04

## 安装

```bash
sudo apt-get install openvpn easy-rsa
```

## 证书和密钥

1. **创建证书目录**：

```bash
mkdir easy-rsa
```

2. **拷贝`easy-rsa`**：

```bash
cp -r /usr/share/easy-rsa/* easy-rsa/
```

3. **进入证书目录**：

```bash
cd easy-rsa
```

4. **编辑`vars`**:

- 找到如下部分，后面的值可以任改 

    改前：
```bash
        export KEY_COUNTRY="US"
        export KEY_PROVINCE="CA"
        export KEY_CITY="SanFrancisco"
        export KEY_ORG="Fort-Funston"
        export KEY_EMAIL="me@myhost.mydomain"
        export KEY_OU="MyOrganizationalUnit"
```

    改后：
```bash
        export KEY_COUNTRY="CN"
        export KEY_PROVINCE="SC"
        export KEY_CITY="ChengDu"
        export KEY_ORG="UESTC"
        export KEY_EMAIL="zyuegege@foxmail.com"
        export KEY_OU="UNIVERSITY"
```

- 找到如下`KEY_NAME`设在为你想要的名字，这里设置成`server`
    
    改前：
```bash
        # X509 Subject Field
        export KEY_NAME="EasyRSA"
```
    改后：
```bash
        # X509 Subject Field
        export KEY_NAME="server"
```

- 生效`vars`中的环境
```bash
source vars   
```

输出：
```
NOTE: If you run ./clean-all, I will be doing a rm -rf on /home/ubuntu/easy-rsa/keys
```

- 执行`./clean-all`

```bash
./clean-all
```

5. **生成证书和密钥**

```bash
./build-ca
```

执行完之后，证书在`keys`目录下：
```
keys
├── ca.crt # CA证书
├── ca.key # 密钥
├── index.txt
└── serial
```

6. **生成OpenVPN服务器证书和密钥**

```bash
./build-key-server server
```

执行完之后，证书在`keys`目录下：
```
keys
├── 01.pem
├── ca.crt # CA证书
├── ca.key # CA私钥
├── index.txt
├── index.txt.attr
├── index.txt.old
├── serial
├── serial.old
├── server.crt   # OpenVPN服务器证书
├── server.csr
└── server.key   # OpenVPN服务器私钥

```

7. **生成Diffe-Hellman参数**

```bash
./build-dh
```

执行完之后，参数在`keys`目录下：

```
keys
├── 01.pem
├── ca.crt    # CA证书
├── ca.key    # CA私钥
├── dh2048.pem  # Diffe-Hellman参数
├── index.txt
├── index.txt.attr
├── index.txt.old
├── serial
├── serial.old
├── server.crt  # OpenVPN服务器证书
├── server.csr
└── server.key  # OpenVPN服务器私钥
```


8. **生成客户端证书**

```bash
source vars
./build-key client
```

9. **证书拷贝**

```bash
# 拷贝服务器证书到/etc/openvpn
sudo cp ca.crt ca.key server.crt server.key ta.key dh2048.pem /etc/openvpn

# 拷贝客户端证书
sudo cp ca.crt client.crt cliet.key ta.key path-to-client-config
```

## 防火墙配置

1. 修改`/etc/sysctl.conf`文件中`net.ipv4.ip_forward`属性的值为`1`:
```bash
net.ipv4.ip_forward=1
```
运行如下命令生效：
```bash
sudo sysctl -p
```

2. 在`/etc/ufw/before.rules`文件中添加`nat`规则:

```bash
# START OPENVPN RULES
# NAT table rules
*nat
:POSTROUTING ACCEPT [0:0]
# Allow traffic from OpenVPN client to eth0(changeto the interface you discovered!)
-A POSTROUTING -s 10.8.0.0/8 -o eth0 -jMASQUERADE
COMMIT
# END OPENVPN RULES
```
**注意**：要加在文件顶部，作为第一条`nat`规则。

3. 在`/etc/default/ufw`中修改转发策略`DROP`为`ACCEPT`:
```bash
DEFAULT_FORWARD_POLICY="ACCEPT"
```

4. 开启端口

```bash
# 开启端口
sudo ufw allow 1194/udp
sudo ufw allow OpenSSH

# 生效防火墙变动
sudo ufw disable
sudo ufw enable
```

## OpenVPN配置

拷贝